# Road safety dataset UK


### Importing libraries 


In [21]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
#from google.colab import drive
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt

### Importing the train and test data from group assignment 

The data which was the output of group assignment is imported here via csv file to 'df_train_clean' and 'df_test_clean' dataframe.

In [22]:
#Loading the datasets.
df_train_clean = pd.read_csv("trainset.csv")
df_test_clean = pd.read_csv("testset.csv")

In [23]:
#This line of code displays all the columns in the dataframe
pd.set_option("display.max_columns", None)

In [24]:
#Check wheather the file is imported correctly
df_train_clean.head()

,longitude,latitude,accident_severity,accident_year_y,vehicle_type,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,driver_imd_decile,driver_home_area_type,Heavy Cargo Vehicle,Heavy Passenger Vehicle,Two-wheeler,Male,Not Known,Child,Elderly,Teen,Young Adult,Heavy oil,Others,Petrol,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,Small town,Urban area
0,-2.177585,53.408598,Serious,2018,Two-wheeler,Male,25.0,Young Adult,999.0,Petrol,11.0,7.0,Urban area,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,-0.092945,51.379843,Slight,2020,Two-wheeler,Male,25.0,Young Adult,124.0,Petrol,3.0,5.0,Urban area,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-1.853969,54.962991,Serious,2020,Two-wheeler,Male,25.0,Young Adult,125.0,Petrol,0.0,4.0,Urban area,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-2.190595,53.853124,Slight,2018,Car,Female,28.0,Young Adult,998.0,Petrol,10.0,2.0,Urban area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.204440,51.488573,Serious,2019,Two-wheeler,Male,17.0,Teen,108.0,Petrol,2.0,8.0,Urban area,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


### Feature selection
Eliminating the categorical columns for which the dummies are created.

In [26]:
x_trainset = df_train_clean.drop(['vehicle_type','sex_of_driver','age_band_of_driver','propulsion_code','driver_imd_decile','driver_home_area_type','accident_year_y'], axis = 1)
x_trainset.head()

,longitude,latitude,accident_severity,age_of_driver,engine_capacity_cc,age_of_vehicle,Heavy Cargo Vehicle,Heavy Passenger Vehicle,Two-wheeler,Male,Not Known,Child,Elderly,Teen,Young Adult,Heavy oil,Others,Petrol,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,Small town,Urban area
0,-2.177585,53.408598,Serious,25.0,999.0,11.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,-0.092945,51.379843,Slight,25.0,124.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-1.853969,54.962991,Serious,25.0,125.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-2.190595,53.853124,Slight,28.0,998.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.204440,51.488573,Serious,17.0,108.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [27]:
#Repeating the same for test data.
y_testset = df_test_clean.drop(['vehicle_type','sex_of_driver','age_band_of_driver','propulsion_code','driver_imd_decile','driver_home_area_type','accident_year_y'], axis = 1)

### Feature scaling
To improvise the performance of Machine Learning algorithms, the values that is ranging in different scales are Standardized. This scaling is acheived by a transformer called 'StandardScaler' that is provided by scikit-learn  

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#Scaling the trainset
trainset_target = x_trainset["accident_severity"].values
trainset_predictors = x_trainset.drop("accident_severity", axis=1)

# fit_transform returns a NumPy array, so we need to put it back 
# into a Pandas dataframe
scaled_vals = scaler.fit_transform(trainset_predictors)
x_trainset = pd.DataFrame(scaled_vals, columns=trainset_predictors.columns)

# put the non-scaled target back in
x_trainset['accident_severity'] = trainset_target

# inspect the data
x_trainset.head()
x_trainset.shape

(422924, 29)

Now use the fitted scaler to scale also the test data, again calling transform() on the fitted scaler:

In [34]:
#Scaling the trainset
testset_target = y_testset["accident_severity"].values
testset_predictors = y_testset.drop("accident_severity", axis=1)

# fit_transform returns a NumPy array, so we need to put it back 
# into a Pandas dataframe
scaled_vals = scaler.fit_transform(testset_predictors)
y_testset = pd.DataFrame(scaled_vals, columns=trainset_predictors.columns)

# put the non-scaled target back in
y_testset['accident_severity'] = testset_target

y_testset.shape

(111775, 29)

### Train models :

The models are trained only on the trainset. Not on the testset.

Seperate arrays for predictors(Xtrain) and target(ytrain) are created 

In [35]:
# drop labels for training set, but keep all others
Xtrain = x_trainset.drop("accident_severity", axis=1)

ytrain = x_trainset["accident_severity"].copy()

In [36]:
#Check whether the array without target variable is created in xtrain
print(Xtrain.shape)
print(x_trainset.shape)

(422924, 28)
(422924, 29)


### Baseline:

The baseline model serves as the baseline for selecting the best ensemble classifier. Any score greater than the score of baseline model is considered a better model

In [37]:
# the number of instances in each class (slight, serious and fatal)
x_trainset['accident_severity'].value_counts()

Slight     329526
Serious     85378
Fatal        8020
Name: accident_severity, dtype: int64

In [38]:
# total size of the training set
x_trainset.shape[0]

422924

The baseline model will output "Slight" for all predictions. Macro-averaging will be used for all the models which invloves evaluating precision, recall and
F-score in each class separately and then average across the classes.

Apply the baseline classifier to the training set.

##### Accuracy measures for the "Slight" variable will be:



Precision = 329526/422924
= 0.7791

Recall = 422924/422924
= 1

F-score = 2/(1/Precision + 1/Recall)
= 0.8758

##### Accuracy measures for the "Serious" variable will be:



Precision = 0

Recall = 0/85378
= 0

F-score = 0



##### Accuracy measures for the "Fatal" variable will be:



Precision = 0

Recall = 0/8020
= 0

F-score = 0



##### Average of all three classes:



Precision = 0.2597



Recall = 0.3333



F-score = 0.2919